# 1. Control inventario de materias primas con apoyo de la ciencia de datos
La empresa X, desde hace 10 años, se dedica a la fabricación de productos para la construcción como: resinas acrílicas, pinturas industriales, recubrimientos para madera, entre otros; y lo hacen procesando las siguientes materias primas: Soda, Xilol, Toluol, 2EHA, Vam, Butil, Ethil, Metil y Estireno. Para la empresa la precisión en el control de inventarios sobre estos insumos es esencial, ya que las materias primas son costosas y altamente sensibles a las condiciones de almacenamiento. La empresa ha experimentado inconsistencias en estos inventarios, afectando la eficiencia de la producción y su estado financiero.

Este proyecto se enfocará en el diseño e implementación de un sistema de control de inventarios que permita la planificación eficiente de las materias primas. Para ello, se cuenta con una base de datos en tiempo real, con información de recetas y adiciones manuales que realizan los operarios en modo semiautomático para la producción.
El objetivo del proyecto será analizar estos datos para encontrar:
Patrones de consumo para encontrar inconsistencias.
Identificar las causas de las variaciones que hay en los inventarios y los reportes de producción.
Desarrollar un modelo predictivo para optimizar la gestión de inventarios, alineando el inventario disponible del almacén con la demanda real de la producción.
Causas y efectos
Las causas del problema incluyen la falta de un sistema de monitoreo continuo, prácticas de compra sin planeación y una previsión de demanda insuficiente. Los efectos de estos problemas se manifiestan en mayores costos, tiempos de producción detenidos y afectación de la rentabilidad de la empresa.

Se deben analizar las variables numericas:
 - Set Point(Solicitudes de materias primas en kg, se abrevia generalmente como SP)
 - Consumo (La metrica del consumo de los sensores de flujo másico, se abrevia como PV)

Se deben analizar variables categoricas como:
 - Solicitudes, partidas o lotes
 - Equipos(los cuales hacen referencia a reactores, tanques o diluidores)
 - Tipo de Solicitud(Receta o semiautomatico)

Se analizan en tiempo ya que son variables que se almacenan en funcion del tiempo
 - TimeStamp

# 2. Cargando Librerias y el conjunto de datos

In [4]:
import pandas as pd
archivo_excel = 'Data.xlsx'
hojas = pd.ExcelFile(archivo_excel).sheet_names


def DataMp(materiaPrima):
    sample_df = pd.read_excel(archivo_excel, sheet_name=materiaPrima,decimal='.', header=0, usecols="C:AO", skiprows=2)

    sample_df['TimeStamp'] = pd.to_datetime(sample_df['TimeStamp'])
    sample_df.set_index('TimeStamp', inplace=True)
    return sample_df


# 4. Preparacion de Datos

In [5]:
def identificar_sp_activo(row, equipo):
    if 'Activo_Receta' + equipo in row:
        if row['Activo_Receta' + equipo]:
            return 'SP Receta ' + equipo
        elif row['Activo_Semi' + equipo]:
            return 'SP Semi ' + equipo
        else:
            return 'Ninguno'
    elif 'Activo_Semi' + equipo in row:
        if row['Activo_Semi' + equipo]:
            return 'SP Semi ' + equipo
    return 'Ninguno'



# Identificar si el bombeo está activo
def CalculoMP(equipo,sample_df):
    
    sample_df['Activo_Semi' + equipo] = sample_df['Estado Bombeo Semi '+ equipo] == 'Active'
    
    if 'Estado Bombeo Receta ' + equipo in sample_df.columns:
        sample_df['Activo_Receta' + equipo] = sample_df['Estado Bombeo Receta '+ equipo] == 'Active'
    

    # Crear una columna que identifique el nuevo SP activo
    # Aplicar la función para identificar el SP activo en cada momento
    # sample_df['SP_Activo' + equipo] = sample_df.apply(identificar_sp_activo, axis=1)
    sample_df['SP_Activo' + equipo] = sample_df.apply(identificar_sp_activo, axis=1, args=(equipo,))


    # Crear una columna con el valor del SP activo en cada momento
    sample_df['Valor_SP_Activo'] = sample_df.apply(
        lambda row: row['SP Receta ' + equipo] if row['SP_Activo' + equipo] == 'SP Receta ' + equipo 
        else (row['SP Semi ' + equipo] if row['SP_Activo' + equipo] == 'SP Semi ' + equipo else 0), 
        axis=1
    )

    # Identificar cambios en el SP activo (cada vez que se activa uno nuevo)
    sample_df['Cambio_SP_Activo'] = sample_df['SP_Activo' + equipo].shift(1) != sample_df['SP_Activo' + equipo]
    sample_df['Consumo R131_1'] = sample_df['Consumo ' + equipo].shift(-1)

    # Crear un número de solicitud único para cada nuevo SP activo
    sample_df['Solicitud'] = sample_df['Cambio_SP_Activo'].cumsum()

    # Agrupar los sample_df por solicitud y obtener los valores finales de SP y PV
    resultadoR131 = sample_df[sample_df['SP_Activo' + equipo] != 'Ninguno'].groupby('Solicitud').agg(
        SP_Activo_Final=('SP_Activo' + equipo, 'last'),
        Valor_SP_Final=('Valor_SP_Activo', 'max'),  # Tomar el máximo de los valores de SP para cada carga
        PV_Final=('Consumo R131_1', 'last')  # Tomar el último PV del grupo
    ).reset_index()
    resultadoR131["Equipo"]=equipo
    return resultadoR131
    # # Sumar todos los valores para cada carga y agregar un total
    # suma_filas = resultadoR131[['Valor_SP_Final', 'PV_Final']].sum().to_frame().T
    # suma_filas['SP_Activo_Final'] = 'Total ' + equipo
    # suma_filas['Solicitud'] = ' '

    # # Concatenar la fila de suma al DataFrame original
    # resultadoSoda_con_sumaR131Soda = pd.concat([resultadoSodaR131, suma_filas], ignore_index=True)
    # # resultadoSoda_con_sumaR131Soda=resultadoSoda

    # # Ordenar el resultadoSoda final para que sea más legible
    # resultado_con_sumaR131Soda = resultadoSod_con_sumaR131Soda[['Solicitud', 'SP_Activo_Final', 'Valor_SP_Final', 'PV_Final']]
    # resultado_con_sumaR131Soda["Equipo"]="R131"
    # # Verificar el resultado
    # return resultado_con_sumaR131Soda


4.1 Materia Prima Soda

In [ ]:
sample_df=DataMp("Soda")
resultadoSodaR131=CalculoMP("R131",sample_df)
resultadoSodaTK131=CalculoMP("TK131",sample_df)
resultadoSodaR141=CalculoMP("TK141",sample_df)
resultadoSodaV142=CalculoMP("V142",sample_df)
resultadoSodaTk141=CalculoMP("TK141",sample_df)
resultadoSodaR221=CalculoMP("R221",sample_df)
resultadoSodaTK221=CalculoMP("TK221",sample_df)

resultadoSoda=pd.concat([resultadoSodaR131,
                      resultadoSodaTK131,
                      resultadoSodaR141,
                      resultadoSodaV142,
                      resultadoSodaTk141,
                      resultadoSodaR221,
                      resultadoSodaTK221])
resultadoSoda["PV_Final"]=pd.to_numeric(resultadoSoda["PV_Final"])
resultadoSoda["Solicitud"]=resultadoSoda["Solicitud"].astype('category')
resultadoSoda

,Solicitud,SP_Activo_Final,Valor_SP_Final,PV_Final,Equipo
0,2,SP Receta R131,1086.929810,1287.476318,R131
1,4,SP Receta R131,1070.750610,1287.889404,R131
2,6,SP Receta R131,1022.079712,1287.872681,R131
3,8,SP Receta R131,1005.200073,1287.883179,R131
4,10,SP Receta R131,988.587219,1287.870483,R131
5,12,SP Receta R131,970.806824,1287.890991,R131
6,14,SP Receta R131,810.816711,1287.893677,R131
7,16,SP Receta R131,793.000000,793.01593,R131
8,18,SP Receta R131,793.000000,793.013672,R131
0,1,SP Semi TK131,638.449890,2500.017334,TK131


4.2 Materia Prima Xilol

In [ ]:
sample_df=DataMp("Xilol")
resultadoXilolR111=CalculoMP("R111",sample_df)
resultadoXilolV115=CalculoMP("V115",sample_df)
resultadoXilolTK112=CalculoMP("TK112",sample_df)
resultadoXilolR221=CalculoMP("R221",sample_df)
resultadoXilolV221=CalculoMP("V221",sample_df)
resultadoXilolTK221=CalculoMP("TK221",sample_df)


resultadoXilol=pd.concat([resultadoXilolR111,
                resultadoXilolV115,
                resultadoXilolTK112,
                resultadoXilolR221,
                resultadoXilolV221,
                resultadoXilolTK221
                ])
resultadoXilol["PV_Final"]=pd.to_numeric(resultadoXilol["PV_Final"])
resultadoXilol["Solicitud"]=resultadoXilol["Solicitud"].astype('category')
resultadoXilol

4.3 Materia Prima Touluol

In [ ]:
sample_df=DataMp("Toluol")
resultadoToluolR111=CalculoMP("R111",sample_df)
resultadoToluolV115=CalculoMP("V115",sample_df)
resultadoToluolTK112=CalculoMP("TK112",sample_df)
resultadoToluolR221=CalculoMP("R221",sample_df)
resultadoToluolV221=CalculoMP("V221",sample_df)
resultadoToluolTK221=CalculoMP("TK221",sample_df)


resultadoToluol=pd.concat([resultadoToluolR111,
                resultadoToluolV115,
                resultadoToluolTK112,
                resultadoToluolR221,
                resultadoToluolV221,
                resultadoToluolTK221
                ])
resultadoToluol["PV_Final"]=pd.to_numeric(resultadoToluol["PV_Final"])
resultadoToluol["Solicitud"]=resultadoToluol["Solicitud"].astype('category')
resultadoToluol

4.4 Materia Prima 2EHA

In [ ]:
sample_df=DataMp("2EHA")

resultado2EHAV115=CalculoMP("V115",sample_df)
resultado2EHAD122=CalculoMP("D122",sample_df)
resultado2EHAV131=CalculoMP("V131",sample_df)
resultado2EHAV141=CalculoMP("V141",sample_df)
resultado2EHAV241=CalculoMP("V241",sample_df)


resultado2EHA=pd.concat([
                resultado2EHAV115,
                resultado2EHAD122,
                resultado2EHAV131,
                resultado2EHAV141,
                resultado2EHAV241
                ])
resultado2EHA["PV_Final"]=pd.to_numeric(resultado2EHA["PV_Final"])
resultado2EHA["Solicitud"]=resultado2EHA["Solicitud"].astype('category')
resultado2EHA

4.5 Materia Prima VAM

In [ ]:
sample_df=DataMp("Vam")

resultadoVamV115=CalculoMP("V115",sample_df)
resultadoVamD122=CalculoMP("D122",sample_df)
resultadoVamV131=CalculoMP("V131",sample_df)
resultadoVamV141=CalculoMP("V141",sample_df)
resultadoVamV241=CalculoMP("V241",sample_df)


resultadoVam=pd.concat([
                resultadoVamV115,
                resultadoVamD122,
                resultadoVamV131,
                resultadoVamV141,
                resultadoVamV241
                ])
resultadoVam["PV_Final"]=pd.to_numeric(resultadoVam["PV_Final"])
resultadoVam["Solicitud"]=resultadoVam["Solicitud"].astype('category')
resultadoVam

4.6 Materia Prima Butil

In [ ]:
sample_df=DataMp("Butil")

resultadoButilV115=CalculoMP("V115",sample_df)
resultadoButilD122=CalculoMP("D122",sample_df)
resultadoButilV131=CalculoMP("V131",sample_df)
resultadoButilV141=CalculoMP("V141",sample_df)
resultadoButilV241=CalculoMP("V241",sample_df)
resultadoButilV221=CalculoMP("V221",sample_df)


resultadoButil=pd.concat([
                resultadoButilV115,
                resultadoButilD122,
                resultadoButilV131,
                resultadoButilV141,
                resultadoButilV241,
                resultadoButilV221
                ])
resultadoButil["PV_Final"]=pd.to_numeric(resultadoButil["PV_Final"])
resultadoButil["Solicitud"]=resultadoButil["Solicitud"].astype('category')
resultadoButil

4.7 Materia Prima Ethil

In [ ]:
sample_df=DataMp("Ethil")

resultadoEthilV115=CalculoMP("V115",sample_df)
resultadoEthilD122=CalculoMP("D122",sample_df)
resultadoEthilV131=CalculoMP("V131",sample_df)
resultadoEthilV141=CalculoMP("V141",sample_df)
resultadoEthilV241=CalculoMP("V241",sample_df)


resultadoEthil=pd.concat([
                resultadoEthilV115,
                resultadoEthilD122,
                resultadoEthilV131,
                resultadoEthilV141,
                resultadoEthilV241
                ])
resultadoEthil["PV_Final"]=pd.to_numeric(resultadoEthil["PV_Final"])
resultadoEthil["Solicitud"]=resultadoEthil["Solicitud"].astype('category')
resultadoEthil

4.8 Materia Prima Metil

In [ ]:
sample_df=DataMp("Metil")

resultadoMetilV115=CalculoMP("V115",sample_df)
resultadoMetilD122=CalculoMP("D122",sample_df)
resultadoMetilV131=CalculoMP("V131",sample_df)
resultadoMetilV141=CalculoMP("V141",sample_df)
resultadoMetilV241=CalculoMP("V241",sample_df)
resultadoMetilV221=CalculoMP("V221",sample_df)


resultadoMetil=pd.concat([
                resultadoMetilV115,
                resultadoMetilD122,
                resultadoMetilV131,
                resultadoMetilV141,
                resultadoMetilV241,
                resultadoMetilV221
                ])
resultadoMetil["PV_Final"]=pd.to_numeric(resultadoMetil["PV_Final"])
resultadoMetil["Solicitud"]=resultadoMetil["Solicitud"].astype('category')
resultadoMetil

4.9 Materia Prima Estireno

In [ ]:
sample_df=DataMp("Estireno")

resultadoEstirenoV115=CalculoMP("V115",sample_df)
resultadoEstirenoD122=CalculoMP("D122",sample_df)
resultadoEstirenoV131=CalculoMP("V131",sample_df)
resultadoEstirenoV141=CalculoMP("V141",sample_df)
resultadoEstirenoV241=CalculoMP("V241",sample_df)
resultadoEstirenoV221=CalculoMP("V221",sample_df)


resultadoEstireno=pd.concat([
                resultadoEstirenoV115,
                resultadoEstirenoD122,
                resultadoEstirenoV131,
                resultadoEstirenoV141,
                resultadoEstirenoV241,
                resultadoEstirenoV221
                ])
resultadoEstireno["PV_Final"]=pd.to_numeric(resultadoEstireno["PV_Final"])
resultadoEstireno["Solicitud"]=resultadoEstireno["Solicitud"].astype('category')
resultadoEstireno